In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [4]:
# load training dataset
train_dataset = torch.load('/content/drive/MyDrive/MakeUoft2024/train_dataset.pt')
train_features = train_dataset['features']
train_targets = train_dataset['targets']

# load test dataset
test_dataset = torch.load('/content/drive/MyDrive/MakeUoft2024/test_dataset.pt')
test_features = test_dataset['features']
test_targets = test_dataset['targets']

In [6]:

train_tensor_dataset = TensorDataset(train_features, train_targets)
test_tensor_dataset = TensorDataset(test_features, test_targets)

# Data Loader
train_loader = DataLoader(train_tensor_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_tensor_dataset, batch_size=512, shuffle=False)

In [14]:

# RNN (GRU) model
class BinaryClassificationRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(BinaryClassificationRNN, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x.shape = (batch_size, sequence_length, input_size)
        out, (hn, cn) = self.rnn(x)
        # Select the output of the last time step
        out = out[:, -1, :]
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

# Hyperparameters
input_size = 18  # Number of features
hidden_size = 128  # Number of features in hidden state
num_layers = 2  # Number of stacked LSTM layers

# Initialize the model
model = BinaryClassificationRNN(input_size, hidden_size, num_layers)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Assuming train_loader is defined and loaded with your dataset
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        # Reshape features to [batch_size, 1, input_size] for single time step
        features = features.unsqueeze(1)  # Adding a sequence length of 1
        labels = labels.unsqueeze(1).float()  # Ensure labels are float for BCELoss

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {loss.item():.4f}')


In [15]:
# Set the model to evaluation mode
model.eval()

# Lists to store predictions and ground truth labels
predictions = []
ground_truth = []

# Disable gradient computation to save memory and computations
with torch.no_grad():
    # Iterate over the test dataset
    for features, labels in test_loader:
        # Reshape features to [batch_size, 1, input_size] for single time step
        features = features.unsqueeze(1)

        # Forward pass
        outputs = model(features)

        # Convert outputs to binary predictions (0 or 1)
        predicted_classes = (outputs > 0.5).float()

        # Append predictions and ground truth labels to the lists
        predictions.extend(predicted_classes.squeeze(1).tolist())
        ground_truth.extend(labels.tolist())

# Convert lists to tensors
predictions = torch.tensor(predictions)
ground_truth = torch.tensor(ground_truth)

# Calculate accuracy
accuracy = (predictions == ground_truth).sum().item() / len(ground_truth)

print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 97.20%


In [18]:
# Save the entire model
torch.save(model, '/content/drive/MyDrive/MakeUoft2024/models/model.pth')

# If you only want to save the model parameters (weights)
torch.save(model.state_dict(), '/content/drive/MyDrive/MakeUoft2024/models/model_weights.pth')

In [ ]:
# import tensorflow as tf
# from torch.autograd import Variable

# # Initialize the PyTorch model
# model = BinaryClassificationRNN(input_size, hidden_size, num_layers)

# # Load the trained PyTorch model weights
# model.load_state_dict(torch.load('model_weights.pth'))

# # Set the model to evaluation mode
# model.eval()

# # Dummy input (batch_size=1, sequence_length=1, input_size)
# dummy_input = Variable(torch.randn(1, 1, input_size))

# # Convert PyTorch model to TensorFlow
# dummy_input_tensorflow = tf.convert_to_tensor(dummy_input)
# traced_model = torch.jit.trace(model, dummy_input_tensorflow)

# # Convert the traced model to TensorFlow Lite format
# converter = tf.lite.TFLiteConverter.from_keras_model(traced_model)
# tflite_model = converter.convert()

# # Save the TensorFlow Lite model to a file
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)